## Generated new data set with systematics

In [1]:
from LHEImport.LHEImport import LHEImport as LHEI
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use(["science", "grid"])
plt.rcParams.update({"font.family":"serif", "font.size":16, "figure.figsize":[20,10]})

In [ ]:
## uncomment this cell to generate the hdf5 files.
data = LHEI('event_files/lhe/ttZ_events_lhapdf_02.lhe')
# data.toHDF5('event_files/hdf5/ttZ_events_02_test', 'particles_limited', limit_events=True)
data.toHDF5('event_files/hdf5/ttZ_events_02', 'particles', limit_events=False)

In [ ]:
# # generating a big list of all data, can then be imported into a pd.DataFrame
# datalist = []
# for i in range(0, len(Data['eventdata'])):
#     event_particles = Data['eventdata'][i]['final_particles']
#     for particle in event_particles:
#         datalist.append(particle.__dict__)

In [ ]:
# df = pd.DataFrame(datalist)

In [ ]:
df = pd.read_hdf('event_files/hdf5/ttZ_events_02.h5', 'particles')
df
# df = pd.read_hdf('event_files/hdf5/ttZ_events_02.h5', 'limit_particles')

## Using scikit-hep/vector testing

In [ ]:
import vector

In [ ]:
df[(df['pdgid']!=21) & (df['barcode']==2)]

In [ ]:
Zs = df.loc[df['pdgid'] ==23]
ts = df.loc[df['pdgid'] ==6]
tbars = df.loc[df['pdgid'] ==-6]

In [ ]:
z_vecs=vector.arr({
    'pt':Zs['pt'].values, 
    'phi':Zs['phi'].values, 
    'eta':Zs['eta'].values, 
    'e':Zs['energy'].values
                })
t_vecs=vector.arr({
    'pt':ts['pt'].values, 
    'phi':ts['phi'].values, 
    'eta':ts['eta'].values, 
    'e':ts['energy'].values
                })
tbar_vecs=vector.arr({
    'pt':tbars['pt'].values, 
    'phi':tbars['phi'].values, 
    'eta':tbars['eta'].values, 
    'e':tbars['energy'].values
                })

In [ ]:
# for particle, particle_string in zip([z_vecs, t_vecs, tbar_vecs], ['z', 't', 't~']):
def single_fourvecplot(particle, particle_string):
    fig, axes = plt.subplots(2,2)
    for i, index, ax in zip(range(4), ["rho", "phi", "eta", "t"], axes.flatten()):
        ax.hist(particle[index], bins = 50, histtype="step")
        if index == "t":
            ax.title.set_text(f"${particle_string}$ : Energy")
        elif index == "rho":
            ax.title.set_text(f"${particle_string}$ : $p_t$")
        else:
            ax.title.set_text(f"${particle_string}$ : $\\{index}$")
    plt.show()

In [ ]:
# fourvecplot(tbar_vecs, 'tbar')

## Plot of $\rho$, $\phi$, $\eta$ and Energy for each particle.

In [ ]:
fig, axes = plt.subplots(2,2)
for param, ax in zip(["rho", "phi", "eta","t"], axes.flatten()):
    for particle, particle_string in zip([z_vecs, t_vecs, tbar_vecs], ["z", "t", "$\\bar{t}$"]):
        ax.hist(particle[param], histtype='step', label=f"{particle_string}", linewidth=2)
        ax.legend(title="Particle", loc="upper right")
        if param == 't':
            ax.title.set_text("Energy")
        elif param == "rho":
            ax.title.set_text("$p_t$")
        else:
            ax.title.set_text(f'$\\{param}$')

In [ ]:
'''
- generate a subset that only has t,t~ and Z
- for particle in dataset plot hist[param]
'''
df_sub = df[(df['pdgid']==23) | (df['pdgid'] ==-6) | (df['pdgid']==6)]
df_sub

In [ ]:
df[df['pdgid']==6].iloc[0].pdgid_string

In [ ]:
fig, axes = plt.subplots(2,2)
for param, ax in zip(["pt", "phi", "eta", "energy"], axes.flatten()):
    for num in [6,-6,23]:
        particle_string = df[df['pdgid']==num].iloc[0].pdgid_latex
        ax.hist(df[df['pdgid']==num][param], histtype='step',label=f"{particle_string}", linewidth=2)
        ax.legend(title="Particle", loc="upper right")
        